# IMFx: Macroeconometric Forecasting 
## Module - 4 Session 3  Workshop Questions
Author: Sabin Poudel 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv("Thailand_M.csv")

In [3]:
df1= df.drop("dateid", axis=1).copy()
df1

,dateid01,p
0,2003-01-01,2.36
1,2003-02-01,2.13
2,2003-03-01,1.82
3,2003-04-01,1.50
4,2003-05-01,1.83
...,...,...
139,2014-08-01,2.12
140,2014-09-01,1.73
141,2014-10-01,1.45
142,2014-11-01,1.24


In [4]:
train = df1.assign(lag1=df1.p.shift(1)).dropna()
train = train.set_index("dateid01")
train =  train[:"2013-12-01"]

In [5]:
test = df1.assign(lag1=df1.p.shift(1)).dropna()
test = test.set_index("dateid01")
test = test["2014-01-01":]



In [6]:
ar1 = SARIMAX(train.p, order=(1, 0, 0), trend = "c").fit()

/home/sabin/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
/home/sabin/.local/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [7]:
print(ar1.summary())

                               SARIMAX Results                                
Dep. Variable:                      p   No. Observations:                  131
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -143.901
Date:                Tue, 15 Feb 2022   AIC                            293.803
Time:                        16:05:10   BIC                            302.428
Sample:                    02-01-2003   HQIC                           297.308
                         - 12-01-2013                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.1963      0.076      2.598      0.009       0.048       0.344
ar.L1          0.9302      0.017     53.516      0.000       0.896       0.964
sigma2         0.5188      0.044     11.863      0.0

In [8]:
pre= ar1.predict(test.index[0], test.index[-1], dynamic=True)
pre.index = pd.to_datetime(pre.index,format="%Y-%m-%d")
pre = pre.to_frame()
type(pre)

pandas.core.frame.DataFrame

In [10]:
#4.16 

test["AR1_Pred"] = pre["predicted_mean"]
test.index = pd.to_datetime(test.index,format="%Y-%m-%d")
test

,p,lag1,AR1_Pred
dateid01,,,
2014-01-01,1.93,1.66,1.740538
2014-02-01,1.95,1.93,1.815458
2014-03-01,2.12,1.95,1.885152
2014-04-01,2.48,2.12,1.949984
2014-05-01,2.62,2.48,2.010294
2014-06-01,2.35,2.62,2.066397
2014-07-01,2.16,2.35,2.118586
2014-08-01,2.12,2.16,2.167134
2014-09-01,1.73,2.12,2.212296


In [12]:
naive=train.p[-1]
naive

1.66

In [13]:
df2= test.copy()

In [14]:
train.index = pd.to_datetime(train.index,format="%Y-%m-%d")

In [15]:
df2 = df2.assign(naive= train.p[-1])

In [16]:
# 4.17

df2= df2.assign(ar1_error = lambda x: x.AR1_Pred- x.p)
df2= df2.assign(naive_error = lambda x: x.naive- x.p)
df2.describe().loc[["mean", "std"]]

,p,lag1,AR1_Pred,naive,ar1_error,naive_error
mean,1.897500,1.984167,2.070273,1.66,0.172773,-0.237500
std,0.566522,0.411725,0.192795,0.00,0.700671,0.566522


In [17]:
df2

,p,lag1,AR1_Pred,naive,ar1_error,naive_error
dateid01,,,,,,
2014-01-01,1.93,1.66,1.740538,1.66,-0.189462,-0.27
2014-02-01,1.95,1.93,1.815458,1.66,-0.134542,-0.29
2014-03-01,2.12,1.95,1.885152,1.66,-0.234848,-0.46
2014-04-01,2.48,2.12,1.949984,1.66,-0.530016,-0.82
2014-05-01,2.62,2.48,2.010294,1.66,-0.609706,-0.96
2014-06-01,2.35,2.62,2.066397,1.66,-0.283603,-0.69
2014-07-01,2.16,2.35,2.118586,1.66,-0.041414,-0.50
2014-08-01,2.12,2.16,2.167134,1.66,0.047134,-0.46
2014-09-01,1.73,2.12,2.212296,1.66,0.482296,-0.07


In [18]:
# 4.19 

df2.query("ar1_error == ar1_error.max() | naive_error == naive_error.max()")




,p,lag1,AR1_Pred,naive,ar1_error,naive_error
dateid01,,,,,,
2014-12-01,0.62,1.24,2.329744,1.66,1.709744,1.04


# 4.20 
As we did in the lecture, calculate the squared error for the AR(1) model over all months of 2014. Then determine the mean squared error (MSE) for AR(1). Fill in the RMSE row as well by taking the square root of your value.

mean square error : $ 1/f{ \sum \limits _{j=1} ^f FE ^2 _{i}}$  
RMSE = $ \sqrt{1/f{ \sum \limits _{j=1} ^f FE ^2 _{i}}}$

 

In [20]:
df2["ar1_square_error"] = df2.eval(df2.ar1_error**2)
df2["naive_error_sq"] = df2.eval(df2.naive_error**2)

print("mse_ar1:", df2.ar1_square_error.mean())
print("rmse_ar1:", np.sqrt(df2.ar1_square_error.mean()))

print("mse_naive:", df2.naive_error_sq.mean())
print("rmse_naive:", np.sqrt(df2.naive_error_sq.mean()))

df2


mse_ar1: 0.47987924068093735
rmse_ar1: 0.692733167013777
mse_naive: 0.35060833333333347
rmse_naive: 0.5921218906047415


,p,lag1,AR1_Pred,naive,ar1_error,naive_error,ar1_square_error,naive_error_sq
dateid01,,,,,,,,
2014-01-01,1.93,1.66,1.740538,1.66,-0.189462,-0.27,0.035896,0.0729
2014-02-01,1.95,1.93,1.815458,1.66,-0.134542,-0.29,0.018102,0.0841
2014-03-01,2.12,1.95,1.885152,1.66,-0.234848,-0.46,0.055154,0.2116
2014-04-01,2.48,2.12,1.949984,1.66,-0.530016,-0.82,0.280917,0.6724
2014-05-01,2.62,2.48,2.010294,1.66,-0.609706,-0.96,0.371742,0.9216
2014-06-01,2.35,2.62,2.066397,1.66,-0.283603,-0.69,0.080431,0.4761
2014-07-01,2.16,2.35,2.118586,1.66,-0.041414,-0.50,0.001715,0.2500
2014-08-01,2.12,2.16,2.167134,1.66,0.047134,-0.46,0.002222,0.2116
2014-09-01,1.73,2.12,2.212296,1.66,0.482296,-0.07,0.232610,0.0049


# 4.21 Calculate the standard forecast error for the AR(1) model.


$ MSE = SE^2 +  BIAS^2 $ 

$BIAS =1/f{ \sum \limits _{j=1} ^f FE} $



In [21]:
bias_ar1 = df2.eval(df2.ar1_error.mean())
bias_naive  = df2.eval(df2.naive_error.mean())

se_ar1 = np.sqrt(df2.ar1_square_error.mean() - bias_ar1**2)
se_nv1 = np.sqrt(df2.naive_error_sq.mean()- (bias_naive**2))

se_ar1<se_nv1


False

# 4.22 

As we did in the lecture, calculate the absolute error for AR(1) (Column J) over all months of 2014. Using these numbers, determine the mean absolute error (MAE) for the AR1 model.

$mean absolute error = 1/f{ \sum \limits  _{j=1} ^f| FE| } $$ 

In [22]:
print("ar1_mae=", np.mean(np.abs(df2.ar1_error)))
print("mae_nv=", np.mean(np.abs(df2.naive_error)))


ar1_mae= 0.5100386002874159
mae_nv= 0.5158333333333335


In [23]:
print("percentage_dff =", ((0.510038-0.515833)/100))

percentage_dff = -5.7949999999999945e-05


# 4.23 
As we did in the lecture, calculate the percentage error and absolute percentage error for AR(1) over all months of 2014. Then determine the mean percentage error (MPE) and the mean absolute percentage error (MAPE) for the AR1 model.

$MPE = 1/f{ \sum \limits  _{j=1} ^f| \frac{FE}{y_t}| } $ 

In [24]:
df2 = df2.assign(mpe_ar1 = (df2.ar1_error/df2.p)*100, mpe_naive = (df2.naive_error/df2.p)*100)
df2 = df2.assign(abs_mpe_ar1 = abs(df2.mpe_ar1), abs_mpe_na = abs(df2.mpe_naive))
print(df2.loc[:,["abs_mpe_ar1","abs_mpe_na"]].apply(np.mean))

abs_mpe_ar1    44.392459
abs_mpe_na     34.551263
dtype: float64
